# Лекция 7. Использование инерциальной навигации в мобильном роботе с использованием MicroPython и симулятора Wokwi

## Введение

Инерциальная навигация — это метод навигации, который использует данные от инерциальных датчиков (акселерометров, гироскопов и иногда магнитометров) для определения положения, скорости и ориентации объекта. В мобильных роботах инерциальная навигация часто используется для отслеживания перемещений, особенно в условиях, где GPS недоступен или неточен.

## 1. Основы инерциальной навигации

### 1.1. Инерциальные датчики

**Акселерометр**: измеряет ускорение по трем осям (X, Y, Z). Может использоваться для определения наклона и перемещения.

**Гироскоп**: измеряет угловую скорость по трем осям. Позволяет определить изменение ориентации.

**Магнитометр**: измеряет магнитное поле Земли, что помогает определить направление (азимут).

### 1.2. Интегрирование данных

**Ускорение → Скорость → Положение**: Интегрируя данные акселерометра, можно получить скорость и положение. Однако, из-за шумов и дрейфа, точность снижается со временем.

**Угловая скорость → Ориентация**: Интегрируя данные гироскопа, можно определить углы наклона (крен, тангаж, рыскание).

### 1.3. Проблемы инерциальной навигации

**Дрейф**: Из-за ошибок интегрирования, положение и ориентация могут "уплывать" со временем.

**Шум**: Датчики имеют шум, который влияет на точность измерений.

***

## 2. Работа с датчиками на MicroPython

### 2.1. Подключение датчиков

Для работы с инерциальными датчиками на MicroPython, можно использовать библиотеку machine для работы с I2C или SPI интерфейсами. Рассмотрим пример подключения MPU6050 (акселерометр + гироскоп) по I2C.


In [ ]:
from machine import I2C, Pin
import time

# Инициализация I2C
i2c = I2C(0, scl=Pin(22), sda=Pin(21), freq=400000)

# Адрес MPU6050
MPU6050_ADDR = 0x68

# Регистры MPU6050
PWR_MGMT_1 = 0x6B
ACCEL_XOUT_H = 0x3B
GYRO_XOUT_H = 0x43

# Включение MPU6050
i2c.writeto_mem(MPU6050_ADDR, PWR_MGMT_1, bytearray([0]))

# Чтение данных акселерометра
def read_accel():
    data = i2c.readfrom_mem(MPU6050_ADDR, ACCEL_XOUT_H, 6)
    ax = (data[0] << 8 | data[1]) / 16384.0
    ay = (data[2] << 8 | data[3]) / 16384.0
    az = (data[4] << 8 | data[5]) / 16384.0
    return ax, ay, az

# Чтение данных гироскопа
def read_gyro():
    data = i2c.readfrom_mem(MPU6050_ADDR, GYRO_XOUT_H, 6)
    gx = (data[0] << 8 | data[1]) / 131.0
    gy = (data[2] << 8 | data[3]) / 131.0
    gz = (data[4] << 8 | data[5]) / 131.0
    return gx, gy, gz

# Основной цикл
while True:
    ax, ay, az = read_accel()
    gx, gy, gz = read_gyro()
    print(f"Accel: {ax}, {ay}, {az} | Gyro: {gx}, {gy}, {gz}")
    time.sleep(0.1)

### 2.2. Фильтрация данных

Для уменьшения шума и дрейфа, данные с датчиков часто фильтруют. Один из популярных методов — комплементарный фильтр или фильтр Калмана.

Пример комплементарного фильтра для определения угла наклона:

In [ ]:
alpha = 0.98  # Коэффициент фильтрации

angle_pitch = 0
angle_roll = 0

while True:
    ax, ay, az = read_accel()
    gx, gy, gz = read_gyro()
    
    # Углы наклона из акселерометра
    accel_pitch = (180 / 3.14159) * atan2(ay, az)
    accel_roll = (180 / 3.14159) * atan2(ax, az)
    
    # Интегрирование данных гироскопа
    dt = 0.1  # Время между измерениями
    gyro_pitch = angle_pitch + gx * dt
    gyro_roll = angle_roll + gy * dt
    
    # Комплементарный фильтр
    angle_pitch = alpha * gyro_pitch + (1 - alpha) * accel_pitch
    angle_roll = alpha * gyro_roll + (1 - alpha) * accel_roll
    
    print(f"Pitch: {angle_pitch}, Roll: {angle_roll}")
    time.sleep(dt)

***
## 3. Симуляция в Wokwi

Wokwi — это онлайн-симулятор для микроконтроллеров, который поддерживает MicroPython. Вы можете симулировать работу робота с инерциальными датчиками.

Перейдите на Wokwi.

Создайте новый проект с микроконтроллером, поддерживающим MicroPython (например, ESP32).

Добавьте виртуальный MPU6050 (или другой IMU датчик) в схему.

### 3.2. Пример кода для симуляции
В Wokwi можно использовать тот же код, что и на реальном устройстве. Просто подключите виртуальный MPU6050 (обратите внимание, что вам нужно подключить ножку SCL к 22-й ножке ESP32, а ножку SDA к 21-й ESP32, не забудьте про шину GND и VCC) и запустите симуляцию. Для того чтобы оценить работу кода можно вручную менять показания MPU6050 - в консоли будут выводится данные об углах в реальном времени.

In [ ]:
from machine import I2C, Pin
import time

# Инициализация I2C
i2c = I2C(0, scl=Pin(22), sda=Pin(21), freq=400000)

# Адрес MPU6050
MPU6050_ADDR = 0x68

# Регистры MPU6050
PWR_MGMT_1 = 0x6B
ACCEL_XOUT_H = 0x3B
GYRO_XOUT_H = 0x43

# Включение MPU6050
i2c.writeto_mem(MPU6050_ADDR, PWR_MGMT_1, bytearray([0]))

# Чтение данных акселерометра
def read_accel():
    data = i2c.readfrom_mem(MPU6050_ADDR, ACCEL_XOUT_H, 6)
    ax = (data[0] << 8 | data[1]) / 16384.0
    ay = (data[2] << 8 | data[3]) / 16384.0
    az = (data[4] << 8 | data[5]) / 16384.0
    return ax, ay, az

# Чтение данных гироскопа
def read_gyro():
    data = i2c.readfrom_mem(MPU6050_ADDR, GYRO_XOUT_H, 6)
    gx = (data[0] << 8 | data[1]) / 131.0
    gy = (data[2] << 8 | data[3]) / 131.0
    gz = (data[4] << 8 | data[5]) / 131.0
    return gx, gy, gz

# Основной цикл
while True:
    ax, ay, az = read_accel()
    gx, gy, gz = read_gyro()
    print(f"Accel: {ax}, {ay}, {az} | Gyro: {gx}, {gy}, {gz}")
    time.sleep(0.1)

***
## 4. Углы Эйлера и кватернионы в управлении мобильным роботом

При управлении мобильным роботом важно учитывать его ориентацию в пространстве. Для описания ориентации используются различные математические подходы, среди которых наиболее популярны углы Эйлера и кватернионы. Эти методы позволяют описать поворот объекта в трехмерном пространстве и широко применяются в робототехнике, авиации, компьютерной графике и других областях.

Рассмотрим, что такое углы Эйлера и кватернионы, как их использовать для управления ориентацией робота.

### 4.1 Углы Эйлера

Углы Эйлера — это три угла, которые описывают поворот объекта вокруг трех осей координат. Обычно используются следующие углы:

Крен (Roll, φ): Поворот вокруг оси X.

Тангаж (Pitch, θ): Поворот вокруг оси Y.

Рыскание (Yaw, ψ): Поворот вокруг оси Z.

Углы Эйлера интуитивно понятны и легко интерпретируются, но имеют ряд ограничений, таких как гимбольный замок (потеря одной степени свободы при определенных углах).

### 4.2 Пример использования углов Эйлера

Предположим, у нас есть данные с гироскопа, и мы хотим вычислить углы Эйлера. Для этого можно интегрировать угловую скорость по времени.

In [ ]:
from math import radians, degrees

# Инициализация углов
roll, pitch, yaw = 0, 0, 0

# Пример данных с гироскопа (в градусах/секунду)
gyro_x, gyro_y, gyro_z = 0.1, 0.2, 0.3  # Примерные значения

# Время между измерениями
dt = 0.1  # 100 мс

# Обновление углов Эйлера
def update_euler_angles(gx, gy, gz, dt):
    global roll, pitch, yaw
    roll += gx * dt
    pitch += gy * dt
    yaw += gz * dt

# Основной цикл
while True:
    update_euler_angles(gyro_x, gyro_y, gyro_z, dt)
    print(f"Roll: {roll}, Pitch: {pitch}, Yaw: {yaw}")
    time.sleep(dt)

### 4.3 Проблема гимбольного замка

Гимбольный замок возникает, когда ось вращения совпадает с одной из осей координат, что приводит к потере одной степени свободы. Это ограничивает применение углов Эйлера в некоторых задачах.

***

## 5. Кватернионы

Кватернионы — это математический инструмент для описания поворотов в трехмерном пространстве. Они состоят из четырех компонентов: q = [w, x, y, z], где:

w — скалярная часть (косинус половины угла поворота).

x, y, z — векторная часть (направление оси поворота).

Кватернионы лишены проблемы гимбольного замка и более эффективны для вычислений, особенно при интерполяции и комбинировании поворотов.

### 5.1 Пример использования кватернионов

Для работы с кватернионами в MicroPython можно использовать библиотеку micropython-ulab или реализовать базовые операции самостоятельно:

In [ ]:
from math import cos, sin, radians

# Инициализация кватерниона (без поворота)
q = [1, 0, 0, 0]

# Функция для обновления кватерниона на основе угловой скорости
def update_quaternion(q, gx, gy, gz, dt):
    w, x, y, z = q
    # Преобразование угловой скорости в радианы
    gx, gy, gz = radians(gx), radians(gy), radians(gz)
    # Вычисление изменения кватерниона
    q_dot = [
        -0.5 * (x * gx + y * gy + z * gz),
        0.5 * (w * gx + y * gz - z * gy),
        0.5 * (w * gy - x * gz + z * gx),
        0.5 * (w * gz + x * gy - y * gx)
    ]
    # Обновление кватерниона
    q[0] += q_dot[0] * dt
    q[1] += q_dot[1] * dt
    q[2] += q_dot[2] * dt
    q[3] += q_dot[3] * dt
    # Нормализация кватерниона
    norm = (q[0]**2 + q[1]**2 + q[2]**2 + q[3]**2) ** 0.5
    q = [qi / norm for qi in q]
    return q

# Основной цикл
while True:
    q = update_quaternion(q, gyro_x, gyro_y, gyro_z, dt)
    print(f"Quaternion: {q}")
    time.sleep(dt)

### 5.2 Преобразование кватернионов в углы Эйлера

Для удобства интерпретации можно преобразовать кватернионы в углы Эйлера.

In [ ]:
from math import atan2, asin

def quaternion_to_euler(q):
    w, x, y, z = q
    # Вычисление углов Эйлера
    roll = atan2(2 * (w * x + y * z), 1 - 2 * (x**2 + y**2))
    pitch = asin(2 * (w * y - z * x))
    yaw = atan2(2 * (w * z + x * y), 1 - 2 * (y**2 + z**2))
    return roll, pitch, yaw

# Пример использования
roll, pitch, yaw = quaternion_to_euler(q)
print(f"Roll: {roll}, Pitch: {pitch}, Yaw: {yaw}")

## 6. Практическое применение в мобильном роботе

### 6.1 Использование углов Эйлера

Углы Эйлера удобны для простых задач, где не требуется высокая точность или сложные повороты. Например, для стабилизации робота по крену и тангажу.

### 6.2 Использование кватернионов

Кватернионы предпочтительны для сложных задач, таких как:

- Плавное управление ориентацией.

- Интерполяция между поворотами.

- Избежание гимбольного замка.


### 6.3 Пример: Управление ориентацией робота

Предположим, у нас есть робот с IMU (акселерометр, гироскоп). Мы можем использовать кватернионы для отслеживания его ориентации и управления движением.

In [ ]:
# Инициализация кватерниона
q = [1, 0, 0, 0]

# Основной цикл
while True:
    # Получение данных с гироскопа
    gx, gy, gz = read_gyro()
    # Обновление кватерниона
    q = update_quaternion(q, gx, gy, gz, dt)
    # Преобразование в углы Эйлера
    roll, pitch, yaw = quaternion_to_euler(q)
    # Управление роботом на основе ориентации
    if abs(pitch) > 0.1:  # Если робот наклонен вперед
        move_forward(10, 5)  # Двигаться вперед
    elif abs(roll) > 0.1:  # Если робот наклонен вбок
        turn_right(10, 5)  # Повернуть направо
    time.sleep(dt)

**Задание для самостоятельной работы:** В лекции приведены примеры получения данных с датчиков (MPU6050) и реализация углов Эйлера и кватернионов. Объедините эти примеры: получайте данные в симуляторе Wokwi, а затем производите вычисления с помощью углов Эйлера и кватернионов, результаты выводите в консоль.

In [ ]:
# Блок для вашего кода

from machine import I2C, Pin
import time

# Инициализация I2C
i2c = I2C(0, scl=Pin(22), sda=Pin(21), freq=400000)

# Адрес MPU6050
MPU6050_ADDR = 0x68

# Регистры MPU6050
PWR_MGMT_1 = 0x6B
ACCEL_XOUT_H = 0x3B
GYRO_XOUT_H = 0x43

# Включение MPU6050
i2c.writeto_mem(MPU6050_ADDR, PWR_MGMT_1, bytearray([0]))
